# Field visualisation using `k3d`

**Note:** If you experience any problems in plotting with `k3d`, please make sure you run the Jupyter notebook in Google Chrome.

There are two ways how a field can be visualised, using:
- `matplotlib`
- `k3d`

`k3d` provides three-dimensional interactive plots of fields inside Jupyter notebook.

Let us say we have a sample, which is an ellipsoid

$$\frac{x^2}{a^2} + \frac{y^2}{b^2} + \frac{z^2}{c^2} <= 1$$

with $a=5\,\text{nm}$, $b=3\,\text{nm}$, and $c=2\,\text{nm}$. The space is discretised into cells with dimensions $(0.5\,\text{nm}, 0.5\,\text{nm}, 0.5\,\text{nm})$. The value of the field at $(x, y, z)$ point is $(-cy, cx, cz)$, with $c=10^{9}$. The norm of the field inside the cylinder is $10^{6}$.

Let us first build that field.

In [1]:
import discretisedfield as df

a, b, c = 5e-9, 3e-9, 2e-9
cell = (0.5e-9, 0.5e-9, 0.5e-9)

mesh =  df.Mesh(p1=(-a, -b, -c), p2=(a, b, c), cell=cell)

def norm_fun(pos):
    x, y, z = pos
    if (x/a)**2 + (y/b)**2 + (z/c)**2 <= 1:
        return 1e6
    else:
        return 0
    
def value_fun(pos):
    x, y, z = pos
    c = 1e9
    return (-c*y, c*x, c*z)

field = df.Field(mesh, dim=3, value=value_fun, norm=norm_fun)

The most basic plot we can show is the plot of all the cells where the value is non-zero. This can be useful, to inspect the domain created, by plotting the norm.

In [2]:
# NBVAL_IGNORE_OUTPUT
field.norm.k3d.nonzero()

Plot(antialias=3, axes=['x', 'y', 'z'], axes_helper=1.0, background_color=16777215, camera=[2, -3, 0.2, 0.0, 0…

Output()

The plot is interactive, so it can be manipulated using a mouse. To change the color of voxels, we can pass the new color via `color` argument.

In [3]:
# NBVAL_IGNORE_OUTPUT
field.norm.k3d.nonzero(color=0x27ae60)

Plot(antialias=3, axes=['x', 'y', 'z'], axes_helper=1.0, background_color=16777215, camera=[2, -3, 0.2, 0.0, 0…

Output()

Next, we can plot a scalar field. For plotting a scalar field, we are using `discretisedfield.Field.k3d_voxels()` method.

In [4]:
# NBVAL_IGNORE_OUTPUT
try:
    field.k3d.scalar()
except ValueError:
    print('Exception raised.')

Exception raised.


An exception was raised because we attempted to plot a vector field using voxels. Therefore, we first need to extract a component of the field. Let us plot the $x$ component.

In [5]:
# NBVAL_IGNORE_OUTPUT
field.x.k3d.scalar()

Plot(antialias=3, axes=['x', 'y', 'z'], axes_helper=1.0, background_color=16777215, camera=[2, -3, 0.2, 0.0, 0…

Output()

However, we can see that the points which we consider to be outside the sample are also plotted. This is because, `discretisedfield.Field.k3d_voxels()` method cannot determine the points where norm is zero from the passed scalar field. Therefore, we need to pass the `norm_field.`

In [6]:
# NBVAL_IGNORE_OUTPUT
field.x.k3d.scalar(filter_field=field.norm, multiplier=1e-6)

Plot(antialias=3, axes=['x', 'y', 'z'], axes_helper=1.0, background_color=16777215, camera=[2, -3, 0.2, 0.0, 0…

Output()

By cascading operations, we can similarly plot the slice of the ellipsoid at $z=0$.

In [7]:
field.plane('x').mesh.region.edges

(5e-10, 6e-09, 4e-09)

In [8]:
# NBVAL_IGNORE_OUTPUT
field.x.plane('z').k3d.scalar(filter_field=field.plane('z').norm)

Plot(antialias=3, axes=['x', 'y', 'z'], axes_helper=1.0, background_color=16777215, camera=[2, -3, 0.2, 0.0, 0…

Output()

To further modify the plot, keyword arguments for `k3d.voxels()` function are accepted. Plese refer to its [documentation](https://k3d-jupyter.readthedocs.io/en/latest/k3d.html#k3d.k3d.voxels).

Next, we can plot the vector field itself.

In [9]:
# NBVAL_IGNORE_OUTPUT
field.k3d.vector()

Plot(antialias=3, axes=['x', 'y', 'z'], axes_helper=1.0, background_color=16777215, camera=[2, -3, 0.2, 0.0, 0…

Output()

By default, points at the discretisation cell centres are plotted together with vectors to help understand the structure of the field. However, they can be deactivated by passing `points=False`.

In [10]:
# NBVAL_IGNORE_OUTPUT
field.k3d.vector(points=False)

Plot(antialias=3, axes=['x', 'y', 'z'], axes_helper=1.0, background_color=16777215, camera=[2, -3, 0.2, 0.0, 0…

Output()

It is difficult to understand the vector field from this plot. By cascading, we can plot its slice at $x=0$.

In [11]:
# NBVAL_IGNORE_OUTPUT
field.plane(x=0).k3d.vector()

Plot(antialias=3, axes=['x', 'y', 'z'], axes_helper=1.0, background_color=16777215, camera=[2, -3, 0.2, 0.0, 0…

Output()

To improve the understanding of the plot, we can now colour the vectors plotted. For that, we need to pass a scalar field, according to which the vectors will be coloured.

In [12]:
# NBVAL_IGNORE_OUTPUT
field.plane(x=0).k3d.vector(color_field=field.x)

Plot(antialias=3, axes=['x', 'y', 'z'], axes_helper=1.0, background_color=16777215, camera=[2, -3, 0.2, 0.0, 0…

Output()

To further modify the plot, keyword arguments for `k3d.vectors()` function are accepted. Plese refer to its [documentation](https://k3d-jupyter.readthedocs.io/en/latest/k3d.html#k3d.k3d.vectors).

### Multiple visualisation on the same plot

Sometimes, it is necessary to show, for example, multiple planes of the sample on the same plot. This can be done by exposing the `k3d.plot` and passing it to different plotting methods. For instance.

In [13]:
# NBVAL_IGNORE_OUTPUT
import k3d
plot = k3d.plot()
field.plane(x=-3e-9).k3d.vector(plot=plot, color_field=field.z)
field.plane(x=0).k3d.vector(plot=plot, color_field=field.z, cmap='hsv')
field.plane(x=3e-9).k3d.vector(plot=plot, color_field=field.z)
plot.display()

Plot(antialias=3, axes=['x\\,\\text{ (nm)}', 'y\\,\\text{ (nm)}', 'z\\,\\text{ (nm)}'], axes_helper=1.0, backg…

Output()

### Plotting regions in the mesh

Different regions can be defined in the mesh. Sometimes it is necessary to visualise those regions in order to make sure they are defined properly.

In [14]:
# NBVAL_IGNORE_OUTPUT
p1 = (0, 0, 0)
p2 = (50, 50, 10)
cell = (5, 5, 5)
mesh = df.Mesh(p1=p1, p2=p2, cell=cell)

region1 = df.Region(p1=(0, 0, 0), p2=(50, 50, 5))
region2 = df.Region(p1=(0, 0, 5), p2=(50, 50, 10))

mesh.subregions = {'region_with_interesting_properties': region1, 'region_with_funny_properties': region2}

mesh.k3d_subregions()

Plot(antialias=3, axes=['x', 'y', 'z'], axes_helper=1.0, background_color=16777215, camera=[2, -3, 0.2, 0.0, 0…

Output()